## Mongo Basic

In [ ]:
# insert movies
db.movies.insert({"id":10,"name": "titanic","Rating": "R"})

# select data
db.movies.find()

# update data
db.movies.update(
{id: 10}, {$set: {Rating: "NR"}}
)

# reove data from movies
db.movies.remove({"Rating": "NR"})

## Mongo Operation On Zip Code Dataset

In [ ]:
mongoimport --db test --collection zips --drop --file zips.json
mongo

In [ ]:
# 切換資料庫
use test

# 存取第一筆資料
db.zips.findOne()

# 使用條件篩選
db.zips.find({state:"MA"})
db.zips.find({state:"MA", , pop: { $gt : 10000}})

## Return States with Populations above 10 Million
# select state, sum(pop) from zips gorup by state having sum(pop) > 10000000
db.zips.aggregate( [
   { $group: { _id: "$state", totalPop: { $sum: "$pop" } } },
   { $match: { totalPop: { $gte: 10000000 } } }
] )

## Return Average City Population by State
## select state, avg(pop) from (  
##     select state, city, sum(pop) as pop from zips group by state, city) zips 
## group by state
db.zips.aggregate( [
   { $group: { _id: { state: "$state", city: "$city" }, pop: { $sum: "$pop" } } },
   { $group: { _id: "$_id.state", avgCityPop: { $avg: "$pop" } } }
] )

## MongoDB UI
- https://www.mongodb.com/download-center?jmp=nav#compass

## 存取Facebook 資訊

In [2]:
ACCESSTOKEN = 'EAACEdEose0cBAEMFFplqcmwHZBqxZCpHfjMwt5Gs2maToSxfAwuP2FRM8WHIEbdeoT6h3sfDh7HFJFieMLnVPlzX2fhzQL38oMEzHXRZABuRTUSNPRhUkyUuVEMOrmhFAq0JXEOJK1oAJjYX8puyyyOYQM2ZBwM6XKhkHUSIKwZDZD'

In [6]:
import requests
res = requests.get('https://graph.facebook.com/v2.7/me/tagged_places?access_token={}'.format(ACCESSTOKEN))
print res.text

{"data":[{"id":"1664781083784462","created_time":"2016-03-09T06:42:31+0000","place":{"id":"182820988487172","location":{"city":"Singapore","country":"Singapore","latitude":25.040897308255,"longitude":121.55372061213,"street":"Singapore Changi Airport","zip":"81"},"name":"\u65b0\u52a0\u5761\u6a1f\u5b9c\u6a5f\u5834"}},{"id":"1664781047117799","created_time":"2016-03-09T06:42:08+0000","place":{"id":"171070619595488","location":{"city":"Ikeda-machi Kitaazumi-gun","country":"Japan","latitude":36.701882048976,"longitude":137.83747383417,"state":"Nagano","street":"\u767d\u99ac\u6751\u516b\u65b9","zip":"399-9301"},"name":"\u767d\u99ac \u516b\u65b9\u5c3e\u6839"}},{"id":"1664780843784486","created_time":"2016-03-09T06:40:18+0000","place":{"id":"150751781627629","location":{"city":"Taipei","country":"Taiwan","latitude":25.035434763842,"longitude":121.51971981538,"street":"No.21, Zhongshan S. Rd., Zhongzheng Dist., Taipei City 100, Taiwan (R.O.C.)","zip":"10048"},"name":"\u4e2d\u6b63\u7d00\u5ff5\u

In [14]:
import json
jd = json.loads(res.text) # 將json 轉變為 字典
place_ary = []
for rec in jd['data']:
    place_name = rec['place']['name']
    place_id = rec['place']['id']
    create_time = rec['created_time']
    place_ary.append({'place_id':place_id ,'place_name': place_name, 'created_time': create_time})

In [15]:
import pandas
df = pandas.DataFrame(place_ary)
df

,created_time,place_id,place_name
0,2016-03-09T06:42:31+0000,182820988487172,新加坡樟宜機場
1,2016-03-09T06:42:08+0000,171070619595488,白馬 八方尾根
2,2016-03-09T06:40:18+0000,150751781627629,中正紀念堂 Chiang Kai-shek Memorial Hall


## 抓取粉絲頁資訊

### 新增應用程式
- https://developers.facebook.com/apps/

In [17]:
ACCESSTOKEN = 'EAACEdEose0cBANzWCdFZAxntjyx2adJImM4lH2x3qektRTMYF5LOw6H1dedE6QCbqfJO16jaPgntb15KSb10F0MZCoysUKv1q6xFVtXCgEUHlkc098owQHIS0477EN23F7eXj8JKJexwQgdSwex7F3ZBveKPI6WA2LTZCCmFZBAZDZD'

In [25]:
fbid = '583984981774221'
fbsecret = '168718a9844764d5739090ecf036b28c'
ACCESSTOKEN = '{}|{}'.format(fbid, fbsecret)

In [26]:
import requests
res = requests.get('https://graph.facebook.com/v2.7/smartloan.tw/posts?access_token={}'.format(ACCESSTOKEN))
jd = json.loads(res.text) # 將json 轉變為 字典
place_ary = []
for rec in jd['data']:
    if 'message' in rec:
        message = rec['message'] 
    #place_name = rec['message']

## 建立與MongoDB 資料庫連線

In [31]:
import pymongo
try:
    conn=pymongo.MongoClient()
    print conn.database_names()
except pymongo.errors.ConnectionFailure as e:
    print "Could not connect to MongoDB: %s" % e

[u'local', u'test']


In [32]:
db= conn['fbdata']
print db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'fbdata')


In [35]:
print conn.database_names()

[u'local', u'test']


In [34]:
coll=db['posts']
print db.collection_names()

[]


In [37]:
import requests
res = requests.get('https://graph.facebook.com/v2.7/smartloan.tw/posts?access_token={}'.format(ACCESSTOKEN))
jd = json.loads(res.text)
coll.insert(jd['data'])

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[ObjectId('57ef6576d18b5e1108f14909'),
 ObjectId('57ef6576d18b5e1108f1490a'),
 ObjectId('57ef6576d18b5e1108f1490b'),
 ObjectId('57ef6576d18b5e1108f1490c'),
 ObjectId('57ef6576d18b5e1108f1490d'),
 ObjectId('57ef6576d18b5e1108f1490e'),
 ObjectId('57ef6576d18b5e1108f1490f'),
 ObjectId('57ef6576d18b5e1108f14910'),
 ObjectId('57ef6576d18b5e1108f14911'),
 ObjectId('57ef6576d18b5e1108f14912'),
 ObjectId('57ef6576d18b5e1108f14913'),
 ObjectId('57ef6576d18b5e1108f14914'),
 ObjectId('57ef6576d18b5e1108f14915'),
 ObjectId('57ef6576d18b5e1108f14916'),
 ObjectId('57ef6576d18b5e1108f14917'),
 ObjectId('57ef6576d18b5e1108f14918'),
 ObjectId('57ef6576d18b5e1108f14919'),
 ObjectId('57ef6576d18b5e1108f1491a'),
 ObjectId('57ef6576d18b5e1108f1491b'),
 ObjectId('57ef6576d18b5e1108f1491c'),
 ObjectId('57ef6576d18b5e1108f1491d'),
 ObjectId('57ef6576d18b5e1108f1491e'),
 ObjectId('57ef6576d18b5e1108f1491f'),
 ObjectId('57ef6576d18b5e1108f14920'),
 ObjectId('57ef6576d18b5e1108f14921')]

In [38]:
print conn.database_names()

[u'fbdata', u'local', u'test']


In [39]:
print db.collection_names()

[u'posts']


## 塞打卡資料

In [42]:
import requests
import json
ACCESSTOKEN = 'EAACEdEose0cBAHdftyDXZBik81UTIuAT3zbS8ZAdxsZCaOIfOuhT4Fm15LFvhZButmQzcQ3PEIUiE5iLapPbdy9wGFbslsVQHN8LkkvGb3fZBsZAfXhdOI37d5bwEXYWkgHWdAGZBmhKtqmPZB9BTrx6Uto6eN9LBLVJFNrAsJjUwQZDZD'
res = requests.get('https://graph.facebook.com/v2.7/me/tagged_places?access_token={}'.format(ACCESSTOKEN))
jd = json.loads(res.text)
coll=db['tagged_places']
coll.insert(jd['data'])

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[ObjectId('57ef66bbd18b5e1108f14922'),
 ObjectId('57ef66bbd18b5e1108f14923'),
 ObjectId('57ef66bbd18b5e1108f14924')]

In [45]:
coll = db['tagged_places']
coll.count()

3

In [46]:
coll = db['posts']
coll.count()

25

## 資料篩選

In [48]:
coll = db['posts']
coll.find_one()

{u'_id': ObjectId('57ef6576d18b5e1108f14909'),
 u'created_time': u'2016-09-29T08:58:13+0000',
 u'id': u'485549298190700_1134806356598321',
 u'message': u'\u7d42\u65bc~ \u4e0d\u964d\u4e86\n\n\u592e\u884c\u81ea\u53bb\u5e74\u7b2c 3 \u5b63\u7406\u76e3\u4e8b\u6703\u8b70\u4ee5\u4f86 4 \u5ea6\u964d\u606f\uff0c\u7e3d\u5171\u5df2\u964d\u606f 2 \u78bc\uff0c\u4eca (29) \u65e5\u53ec\u958b 105 \u5e74\u7b2c 3 \u5b63\u7406\u76e3\u4e8b\u806f\u5e2d\u6703\u8b70\uff0c\u6c7a\u8b70\u7dad\u6301\u5229\u7387\u6309\u5175\u4e0d\u52d5...'}

In [51]:
cur=coll.find({'created_time':{'$gt':'2016-09-13'}})
for rec in cur:
    print rec['created_time']

2016-09-29T08:58:13+0000
2016-09-27T02:49:20+0000
2016-09-23T00:56:14+0000
2016-09-22T12:01:17+0000
2016-09-21T03:21:25+0000
2016-09-21T03:17:30+0000
2016-09-18T13:46:06+0000
2016-09-17T05:40:05+0000
2016-09-14T01:31:10+0000
2016-09-13T06:19:58+0000
2016-09-13T02:48:01+0000


In [53]:
cur=coll.find({'message':{'$regex':'房貸'}})
for rec in cur:
    message = rec['message']

In [60]:
cur=coll.find({'message':{'$regex':'房貸'}}).sort('created_time').limit(1)
for doc in cur:
    print doc['id']

485549298190700_1098877666857857


In [58]:
cur=coll.find({'message':{'$regex':'房貸'}}).sort('created_time', pymongo.DESCENDING).limit(1)
for doc in cur:
    print doc['created_time']

2016-09-22T12:01:17+0000


In [61]:
cur = coll.find({"id":"485549298190700_1098877666857857"})
for doc in cur:
    print doc


{u'created_time': u'2016-08-19T00:00:00+0000', u'message': u'\u82e5\u300c\u9752\u5e74\u5b89\u5fc3\u6210\u5bb6\u8cb8\u6b3e\u300d\u771f\u7684\u558a\u505c\uff0c\u90a3\u5c07\u6703\u628a\u5df2\u51b7\u5230\u4e0d\u884c\u7684\u623f\u5e02\u76f4\u63a5\u9001\u9032\u51b7\u51cd\u5eab ...\n\n\u7562\u7adf\u300c\u9752\u5e74\u5b89\u5fc3\u6210\u5bb6\u8cb8\u6b3e\u300d\u7121\u8ad6\u662f\u5229\u7387\u3001\u984d\u5ea6\u3001\u5e74\u9650\u90fd\u5e7e\u4e4e\u53ef\u4ee5\u6eff\u8db3\u9996\u6b21\u8cfc\u5c4b\u7684\u6c11\u773e\u4f7f\u7528\uff0c\u4e00\u65e6\u558a\u505c\uff0c\u5c0d\u73fe\u5b58\u50c5\u5269\u81ea\u4f4f\u9700\u6c42\u7684\u75b2\u5f31\u623f\u5730\u7522\u5e02\u5834\u7121\u7591\u662f\u96ea\u4e0a\u52a0\u971c\u554a~  \u5c0d\u65bc\u6eff\u624b\u73fe\u91d1\u7121\u6cd5\u653e\u6b3e\u51fa\u53bb\u7684\u9280\u884c\u76f8\u4fe1\u4e5f\u4e0d\u662f\u4e00\u4ef6\u597d\u4e8b... \n\nhttp://goo.gl/mlwks4\n\n\uff03\u9752\u5e74\u5b89\u5fc3\u6210\u5bb6\u8cb8\u6b3e \uff03\u653f\u5e9c\u512a\u60e0\u8cb8\u6b3e \n\n<\u5ee3\u7de8> \u73f

In [62]:
result=coll.update_one({"id":"485549298190700_1098877666857857"},{"$set":{"tag":"貸款"},"$currentDate":{"lastModified":True}})

In [68]:
cur = coll.find({"id":"485549298190700_1098877666857857"})
for doc in cur:
    print doc['tag'], doc['lastModified']

貸款 2016-10-01 07:52:33.823000


In [71]:
cur = coll.find({"id":"485549298190700_1098877666857857"})
for doc in cur:
    print type(doc['created_time'])

<type 'unicode'>


In [75]:
from datetime import datetime
a = '2016-08-19T00:00:00'
datetime.strptime(a, '%Y-%m-%dT%H:%M:%S')

datetime.datetime(2016, 8, 19, 0, 0)

In [78]:
for obj in coll.find():
    time=datetime.strptime(obj['created_time'],"%Y-%m-%dT%H:%M:%S+0000")
    coll.update({'_id':obj['_id']},{'$set':{'created_time':time}})

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  app.launch_new_instance()


In [80]:
cur = coll.find({"id":"485549298190700_1098877666857857"})
for doc in cur:
    print type(doc['created_time'])

<type 'datetime.datetime'>


## Aggregation 範例 - 計算每月有發多少文章

In [82]:
# select month(created_time), yaer(created_time), sum(1)
# from posts group by month(created_time), year(created_time)

egg=coll.aggregate([
        {'$project':{'month':{'$month':"$created_time"},'year':{'$year':"$created_time"}}},
        {'$group':{'_id':{'month':'$month','year':'$year'},'number':{'$sum':1}}},
    ])

for doc in egg:
    print(doc)

{u'_id': {u'month': 8, u'year': 2016}, u'number': 6}
{u'_id': {u'month': 9, u'year': 2016}, u'number': 19}


## 文章刪除與Collection 捨棄

In [83]:
coll.delete_many({'message':{'$regex':'房貸'}})

In [87]:
for doc in coll.find():
    message =  doc['message']
    #print message

KeyError: 'message'

In [88]:
coll.count()

15

In [89]:
coll.drop()

In [90]:
coll.count()

0

In [91]:
print conn.database_names()

[u'fbdata', u'local', u'test']


In [92]:
print db.collection_names()

[u'tagged_places']


In [93]:
coll2 = db['tagged_places']
coll2.drop()

In [94]:
conn.database_names()

[u'local', u'test']